In [ ]:
vlib =  c("tidyverse", "ggpubr", "arrow", "ComplexHeatmap", "Seurat",
         "tidyseurat", "enrichR", "ComplexHeatmap", , "enrichR",
         "presto", "patchwork", "ggpmisc", "ggrepel", "broom", "viridis")
lapply(vlib, require, character.only = TRUE, quietly = TRUE) |> suppressMessages()

base_dir = "."
setwd(base_dir)

saveRDS.gz <- function(object, file, threads = 4) {
  con <- pipe(paste0("pigz -p", threads, " > ", file), "wb")
  saveRDS(object, file = con)
  close(con)
}
readRDS.gz <- function(file, threads = parallel::detectCores()) {
  con <- pipe(paste0("pigz -d -c -p", threads, " ", file))
  object <- readRDS(file = con)
  close(con)
  return(object)
}

## figure 4D
due to large size of SCENIC GRN matrix, we provide a DE-GRN matrix for visualization.

In [ ]:
df_diff_grn = read_delim("assets/SCENIC/scenic_DE_GRN.txt.gz")

In [ ]:
c_cluster = "CD8"
df_map_cis = read_delim("assets/02_mashr/joined_all_results.txt.gz")
df_eGene = df_map_cis %>% filter(lfsr < 0.05)
c_eGene = df_eGene %>% filter(grepl(c_cluster, condition)) %>% distinct(phenotype_id) %>% pull()
c_noteGene = df_map_cis %>%
  filter(!phenotype_id %in% c_eGene) %>%
  filter(grepl(c_cluster, condition)) %>%
  distinct(phenotype_id) %>%
  pull()

In [ ]:
adj_mtx = read_delim("ssets/SCENIC/adj.csv")

df_cd8_regulone_enriched = adj_mtx %>%
  mutate(eGene = ifelse(target %in% c_eGene, "eGene",
                        ifelse(target %in% c_noteGene, "not_eGene", NA))) %>%
  na.omit() %>%
  group_by(TF, eGene) %>%
  summarise(n = n()) %>%
  pivot_wider(names_from = "eGene", values_from = n) %>%
  filter(TF %in% str_sub(c_cd8_scenic_regulon, 1, -4)) %>%
  mutate(ratio = eGene/(eGene + not_eGene)) %>%
  arrange(-ratio)

In [ ]:
pSCENIC_DEGRN = df_diff_grn %>%
  filter(group == "PD base CD8") %>%
  mutate(feature = str_sub(feature, 1, -4)) %>%
  left_join(df_cd8_regulone_enriched, by = c("feature" = "TF"))  %>%
  arrange(padj) %>%
  mutate(label = ifelse((abs(logFC) > 0.005), feature, "")) %>%
  na.omit() %>%
  ggplot(aes(x = logFC, y = -log10(padj))) +
  geom_vline(xintercept = c(-0.005, 0.005), color = "grey90") +
  geom_hline(yintercept = c(-log10(1e-5)), color = "grey90")+
  geom_point() +
  geom_text_repel(aes(label = label)) +
  theme_pubr()

options(repr.plot.width = 4, repr.plot.height = 4, repr.plot.res = 300)
pSCENIC_DEGRN
ggsave("figure_prep/pSCENIC_DEGRN.pdf",pSCENIC_DEGRN, width = 4, height = 4, dpi = 300)

## figure 4E

In [ ]:
fname_module_score = "analysis/assets/wgcna/module_gene_scored.CD8.RNA.txt.gz"
df_module_score = read_delim(fname_module_score, delim = "\t")
list_adj_moduleScore = left_join(adj_mtx, df_module_score, by = c("target" = "gene_name")) %>%
  filter(TF %in% str_sub(c_cd8_scenic_regulon, 1, -4)) %>%
  group_by(TF, color) %>%
  group_split()

In [ ]:
list_res_tidy = vector(mode = "list", length = length(list_adj_moduleScore))
list_res_glance = vector(mode = "list", length = length(list_adj_moduleScore))
for (i in seq_along(list_adj_moduleScore)){
  tryCatch({
    each_lm = list_adj_moduleScore[[i]] %>%
      lm(importance ~ value, data = .)
    list_res_tidy[[i]] = each_lm %>%
      broom::tidy() %>%
      mutate(TF = list_adj_moduleScore[[i]]$TF[1],
             color = list_adj_moduleScore[[i]]$color[1])
    list_res_glance[[i]] = each_lm %>%
      broom::glance() %>%
      mutate(TF = list_adj_moduleScore[[i]]$TF[1],
             color = list_adj_moduleScore[[i]]$color[1])
  },
  error = function(e) {
    list_res_tidy[[i]] = NA
    list_res_glance[[i]] = NA
  }
  )
}

df_res_tidy = list_res_tidy %>% bind_rows()
df_res_glance = list_res_glance %>% bind_rows()

In [ ]:
df_diff_grn_significant = df_diff_grn %>%
  filter(feature %in% c_cd8_scenic_regulon,
    group == "PD base CD8",
    abs(logFC) > 0.005
  )

p_heatmap_padj = df_res_tidy %>% 
  mutate(p.adj = p.adjust(p.value, method= "BH")) %>%
  filter(term == "value",
         TF %in% str_sub(df_diff_grn_significant$feature, 1, -4)) %>%
  select(p.adj, TF, color) %>%
  mutate(p.adj = ifelse(p.adj < 0.05, -log10(p.adj), 
                        ifelse(is.na(p.adj) == TRUE, 0, 0))) %>%
  pivot_wider(names_from = TF, values_from = p.adj, values_fill = 0) %>%
  as.data.frame() %>%
  column_to_rownames("color") %>%
  t() %>%
  Heatmap(
    heatmap_legend_param = list(
      title = "p.adj"
    ),
    col = colorRamp2(c(0, -log10(0.05), 10), c("#53799d", "white", "#E23222")),
    column_dend_height = unit(0.5, "cm"),
    row_dend_width = unit(0.5, "cm")
  )
  p_heatmap_padj

pdf("figure_prep/pSCENIC_heatmap_padj.pdf", width = 5, height = 5)
p_heatmap_padj
dev.off()

## figure 4F

In [ ]:
df_cd8_MEs = read_delim("analysis/assets/wgcna/CD8_MEs.txt.gz")
df_scenic_module_merged = left_join(df_cd8_MEs, seurat_obj_aucmtx) %>%
  select(cellID, anno_l1, anno_l2, brown, `TBX21(+)`, `EOMES(+)`)